# Projet 3 : développer à partir de l’expression de gènes (puce à ADN) un outil de diagnostic, capable de prédire plusieurs types de cancer.

Etudiants :
- Mattei Simon
- Mercado Pablo

## Introduction
### Un rappel sur ce qu'est le cancer
Le cancer est la prolifération de cellules anormales dans un tissu. Elles peuvent utiliser le système sanguin pour voyager et aller affecter d'autres tissus. Certains cancers sont d'origine génétique et ils existent plusieurs facteurs de risques (consommation excessive de tabac et alcool, exposition au soleil, obésité...). Manger une [pomme](https://pubmed.ncbi.nlm.nih.gov/23091184/), faites de l'exercice et manger sainement pour éviter le cancer!

Bien que l'âge ne soit pas la cause directe du cancer, on observe un taux d'incidence plus élevée chez les personnes agées. Avec le vieillissement de la population, la recherche du cancer est un domaine qui pourrait sauver beaucoup de vies.

### Types de cancer
On peut classer les [cancers](https://fr.wikipedia.org/wiki/Cancer) par les organes (et tissus) qu'ils touchent :
*    le cancer du sein
*    le cancer du côlon
*    le cancer du pancréas
*    le cancer de la prostate
*    le cancer du cerveau
*    le myélome multiple : cancer de la moelle osseuse
*    les leucémies
*    le sarcome de Kaposi : cancer des vaisseaux sanguins
*    les lymphomes
*    le cancer des testicules
*    le cancer des poumons
*    le cancer des reins
*    le cancer du foie

### Panel NCI60
Le dataset utilisé prend des [lignées cellulaires (cell line)](https://fr.wikipedia.org/wiki/Lign%C3%A9e_cellulaire_(m%C3%A9decine)), c-à-d des cellules pouvant théoriquement se diviser à l'infini.

### Comment chercher un remède à un cancer et pourquoi avec l'expression des gènes?


In [7]:
import numpy as np

In [8]:
from Bio import Geo

def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records

records = load_geo('GDS4296.soft')

In [9]:
while True:
    try:
        nr = next(records)
        #print(nr)
        print('\n---------------------- ENTITY_ATTRIBUTES:------------------------------\n')
        print(nr.entity_attributes)
    except StopIteration:
        print("done reading")
        break


---------------------- ENTITY_ATTRIBUTES:------------------------------

{'Database_name': 'Gene Expression Omnibus (GEO)', 'Database_institute': 'NCBI NLM NIH', 'Database_web_link': 'http://www.ncbi.nlm.nih.gov/geo', 'Database_email': 'geo@ncbi.nlm.nih.gov', 'Database_ref': 'Nucleic Acids Res. 2005 Jan 1;33 Database Issue:D562-6'}

---------------------- ENTITY_ATTRIBUTES:------------------------------

{'dataset_title': 'NCI-60 cancer cell line panel', 'dataset_description': 'Analysis of cell lines from 9 different cancer tissue of origin types (Breast, Central Nervous System, Colon, Leukemia, Melanoma, Non-Small Cell Lung, Ovarian, Prostate, and Renal) from the NCI-60 panel. Results provide insight into molecular mechanisms underlying the various cancer types.', 'dataset_type': 'Expression profiling by array', 'dataset_pubmed_id': '19584232', 'dataset_platform': 'GPL570', 'dataset_platform_organism': 'Homo sapiens', 'dataset_platform_technology_type': 'in situ oligonucleotide', 'da


---------------------- ENTITY_ATTRIBUTES:------------------------------

{'dataset_table_begin': '', 'dataset_table_end': ''}
done reading


Extrayons les données utiles: niveaux d'expressions des conditions et sources des conditions.

In [20]:
records = load_geo('GDS4296.soft')

# description of each condition (cell type)
id_cond_tissue = {}

diseases = set()
#disease_with_details = set()

# loop over records
for r in records:
    rea = r.entity_attributes

    # condition for cell type/tissue/disease state
    if 'subset_dataset_id' in rea:
        sample_list = rea['subset_sample_id'].split(',')
        desc = rea['subset_description']

        if rea['subset_type'] == 'tissue':
            for sample in sample_list:
                id_cond_tissue[sample] = desc
                diseases.add(desc)

        # si l'on veut plus de détail sur la variété de cancer
        # e.g. https://fr.wikipedia.org/wiki/Astrocytome
        #if rea['subset_type'] == 'disease state':
            #for sample in sample_list:
                #id_cond_disease[sample] = desc
                #disease_with_details.add(desc)

    if 'dataset_table_begin' in rea:
        data = r.table_rows

# converts data to an array (more practical to handle)
data = np.array(data)
print(len(id_cond_tissue))

174


In [21]:
print(diseases)
id_cond_tissue

{'non-small cell lung', 'melanoma', 'leukemia', 'ovarian', 'CNS', 'prostate', 'renal', 'breast', 'colon'}


{'GSM803621': 'breast',
 'GSM803680': 'breast',
 'GSM803739': 'breast',
 'GSM803622': 'breast',
 'GSM803681': 'breast',
 'GSM803740': 'breast',
 'GSM803623': 'breast',
 'GSM803682': 'breast',
 'GSM803741': 'breast',
 'GSM803625': 'breast',
 'GSM803684': 'breast',
 'GSM803743': 'breast',
 'GSM803673': 'breast',
 'GSM803731': 'breast',
 'GSM803788': 'breast',
 'GSM803627': 'CNS',
 'GSM803686': 'CNS',
 'GSM803745': 'CNS',
 'GSM803628': 'CNS',
 'GSM803687': 'CNS',
 'GSM803746': 'CNS',
 'GSM803629': 'CNS',
 'GSM803688': 'CNS',
 'GSM803747': 'CNS',
 'GSM803630': 'CNS',
 'GSM803689': 'CNS',
 'GSM803748': 'CNS',
 'GSM803631': 'CNS',
 'GSM803690': 'CNS',
 'GSM803749': 'CNS',
 'GSM803632': 'CNS',
 'GSM803691': 'CNS',
 'GSM803750': 'CNS',
 'GSM803633': 'colon',
 'GSM803692': 'colon',
 'GSM803751': 'colon',
 'GSM803634': 'colon',
 'GSM803693': 'colon',
 'GSM803752': 'colon',
 'GSM803635': 'colon',
 'GSM803694': 'colon',
 'GSM803753': 'colon',
 'GSM803636': 'colon',
 'GSM803695': 'colon',
 'GSM8037

In [19]:
print(data[3,5]) # élément de data à l'intersection de la 3ème ligne et 5ème colonne
print(data[4,:]) # ligne 4 de data = un gène en particulier
print(data[:,2]) # colonne 7 de data = une condition en particulier !! imprimée sur une ligne, mais c'est une colonne !!
print(len(data[:,1]))

4.96283
['121_at' 'PAX8' '7.25647' '7.34984' '7.03978' '7.23067' '7.63878'
 '7.45663' '7.60143' '7.56602' '7.27995' '7.72512' '8.09685' '7.8255'
 '7.78146' '7.89176' '7.5984' '7.50742' '7.60422' '7.5756' '7.59214'
 '7.64121' '7.72563' '7.38292' '7.31098' '7.1515' '7.61873' '7.72867'
 '7.76086' '7.69995' '8.60788' '9.07909' '7.57886' '7.80053' '7.60925'
 '7.23072' '7.44112' '7.54419' '7.2752' '7.51679' '7.43727' '7.35637'
 '7.37582' '7.44042' '7.01289' '7.19171' '7.09156' '7.41675' '7.37547'
 '7.55363' '7.26298' '7.31134' '7.34165' '7.40834' '7.39202' '7.42262'
 '7.69765' '7.90455' '8.16485' '7.51886' '7.71048' '7.81114' '7.53439'
 '7.84801' '7.65642' '7.56078' '7.56565' '7.51897' '7.68262' '7.90417'
 '7.73797' '7.47687' '7.72709' '7.37865' '7.75278' '7.83489' '7.84638'
 '9.29607' '9.13335' '8.86856' '8.55122' '8.67774' '8.5656' '9.83422'
 '10.2651' '9.72584' '9.10971' '9.01537' '9.70878' '9.57623' '9.64363'
 '7.80508' '8.03154' '7.96988' '9.7424' '9.69476' '9.69079' '9.71988'
 '9.75847

<b>Séparation en données de test et train</b>

In [27]:
import random

# init dictionnary with disease - empty list
diseases_cond = {}
for disease in diseases:
    diseases_cond[disease] = []

# add every condition to dictionnary
for cond in id_cond_tissue:
    diseases_cond[id_cond_tissue[cond]].append(cond)
    
# shuffle conditions and add them to train/test set
test_set = []
train_set = []
for disease in diseases_cond:
    random.shuffle(diseases_cond[disease])
    idx_sep = 


    
diseases_cond

{'non-small cell lung': ['GSM803710',
  'GSM803769',
  'GSM803653',
  'GSM803654',
  'GSM803655',
  'GSM803771',
  'GSM803765',
  'GSM803708',
  'GSM803767',
  'GSM803714',
  'GSM803713',
  'GSM803651',
  'GSM803707',
  'GSM803711',
  'GSM803772',
  'GSM803712',
  'GSM803649',
  'GSM803652',
  'GSM803715',
  'GSM803770',
  'GSM803656',
  'GSM803709',
  'GSM803768',
  'GSM803650',
  'GSM803766',
  'GSM803648'],
 'melanoma': ['GSM803785',
  'GSM803729',
  'GSM803672',
  'GSM803668',
  'GSM803786',
  'GSM803669',
  'GSM803744',
  'GSM803723',
  'GSM803685',
  'GSM803666',
  'GSM803724',
  'GSM803781',
  'GSM803726',
  'GSM803782',
  'GSM803784',
  'GSM803671',
  'GSM803787',
  'GSM803667',
  'GSM803670',
  'GSM803783',
  'GSM803730',
  'GSM803727',
  'GSM803665',
  'GSM803725',
  'GSM803626',
  'GSM803728'],
 'leukemia': ['GSM803616',
  'GSM803735',
  'GSM803736',
  'GSM803674',
  'GSM803733',
  'GSM803677',
  'GSM803737',
  'GSM803615',
  'GSM803734',
  'GSM803617',
  'GSM803676',
  'GSM